# Lab | Langchain Evaluation

## Intro

Pick different sets of data and re-run this notebook. The point is for you to understand all steps involve and the many different ways one can and should evaluate LLM applications.

What did you learn? - Let's discuss that in class

## LangChain: Evaluation

### Outline:

* Example generation
* Manual evaluation (and debuging)
* LLM-assisted evaluation

In [1]:
from google.colab import userdata
import os

OPENAI_API_KEY  = userdata.get('OPENAI_API_KEY_Ironhack')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [2]:
!pip install langchain
!pip install langchain-openai
!pip install langsmith
!pip install ragas
!pip install recordclass

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.6/983.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.8/355.8 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.4/328.4 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.5/91.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install --quiet langchain_huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 68.8 MB/s eta 0:00:00


### Example 1

#### Create our QandA application

In [4]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.llms import OpenAI
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import CSVLoader, TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.chains import LLMChain


In [6]:
file = '/content/OutdoorClothingCatalog_1000.csv' #adjusted path to data to access it from colab
loader = CSVLoader(file_path=file)
data = loader.load()

In [8]:
!pip install --quiet --upgrade --force-reinstall sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 82.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 100.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 MB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.6/402.6 kB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 117.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.5/705.5 kB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 119.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [7]:
!pip install --quiet docarray

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.2/270.2 kB 5.4 MB/s eta 0:00:00


**2. Create an index to enable efficient similarity search**

**Embedding**: use the HuggingFaceEmbeddings all-MiniLM-L6-v2 model to convert data into numerical vectors that capture semantic meaning and generate text embeddings

**Vectorstore**: DocArrayInMemorySearch stores these embeddings in memory, allowing for fast retrieval.

**Index**: VectorstoreIndexCreator combines these components to create an index to find data points that are semantically similar to a given query.

In [10]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2", model_kwargs = {'device': 'cuda'})
).from_loaders([loader])

/usr/local/lib/python3.10/dist-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


**3. Set up the question-answering system using LangChain**

In [11]:
llm = ChatOpenAI(temperature = 0.0)
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff", #the entire relevant document context will be provided to the language model
    retriever=index.vectorstore.as_retriever(),  #retrieve relevant documents based on query similarity using the index previously created
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

#### Coming up with test datapoints

In [12]:
data[10]

Document(metadata={'source': '/content/OutdoorClothingCatalog_1000.csv', 'row': 10}, page_content=": 10\nname: Cozy Comfort Pullover Set, Stripe\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.\n\nSize & Fit\n- Pants are Favorite Fit: Sits lower on the waist.\n- Relaxed Fit: Our most generous fit sits farthest from the body.\n\nFabric & Care\n- In the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features\n- Relaxed fit top with raglan sleeves and rounded hem.\n- Pull-on pants have a wide elastic waistband and drawstring, side pockets and a modern slim leg.\n\nImported.")

In [13]:
data[11]

Document(metadata={'source': '/content/OutdoorClothingCatalog_1000.csv', 'row': 11}, page_content=': 11\nname: Ultra-Lofty 850 Stretch Down Hooded Jacket\ndescription: This technical stretch down jacket from our DownTek collection is sure to keep you warm and comfortable with its full-stretch construction providing exceptional range of motion. With a slightly fitted style that falls at the hip and best with a midweight layer, this jacket is suitable for light activity up to 20° and moderate activity up to -30°. The soft and durable 100% polyester shell offers complete windproof protection and is insulated with warm, lofty goose down. Other features include welded baffles for a no-stitch construction and excellent stretch, an adjustable hood, an interior media port and mesh stash pocket and a hem drawcord. Machine wash and dry. Imported.')

In [14]:
data[400]

Document(metadata={'source': '/content/OutdoorClothingCatalog_1000.csv', 'row': 400}, page_content=': 400\nname: Mountain Crest Insulated Pullover\ndescription: We took the classic anorak look and combined it with the exceptional warmth of PrimaLoft® insulation - the perfect throw-on-and-go for transitional weather. Light activity is comfortable up to 45° and Moderate activity is comfortable up to -5°.\n\nSize & Fit: Slightly Fitted. Best with lightweight layer. Falls at hip.\n\nWhy We Love It: Inspired by our archives, we designed this insulated pullover hoodie to have all the features you love about our classic anorak, plus the innovative, lightweight warmth of PrimaLoft insulation. We love throwing it on for morning walks with the dog, exploring a new trail or simply stepping out for the weekend.\n\nFabric & Care: Weather-resistant, lightweight 100% polyester. Packed with 40 grams of ultralight, ultrawarm PrimaLoft insulation. Machine wash and dry.\n\nAdditional Features: Elastic ho

#### Hard-coded examples

In [15]:
from langchain.prompts import PromptTemplate

In [16]:
from langchain.prompts import PromptTemplate
from langchain.schema import BaseOutputParser
from pydantic import BaseModel, Field

examples = [
    {
        "query": "Do the Cozy Comfort Pullover Set\
        have side pockets?",
        "answer": "Yes"
    },
    {
        "query": "What collection is the Ultra-Lofty \
        850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]

# Define the prompt template
prompt_template = PromptTemplate(
    input_variables=["query"],
    template="Examples:\n"
             "1. Query: Do the Cozy Comfort Pullover Set have side pockets?\n"
             "   Answer: Yes\n"
             "2. Query: What collection is the Ultra-Lofty 850 Stretch Down Hooded Jacket from?\n"
             "   Answer: The DownTek collection\n"
             "Query: {query}\n"
             "Answer:"
)

# Define the output model
class Answer(BaseModel):
    answer: str = Field(description="The answer to the query")

# Create the output parser
class AnswerOutputParser(BaseOutputParser):
    def parse(self, text: str) -> Answer:
        # Split the response to get the answer
        answer = text.strip().split("Answer:")[-1].strip()
        return Answer(answer=answer)

# Initialize the LLM
# llm = OpenAI()
llm = ChatOpenAI()

# Create the LLMChain
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    output_parser=AnswerOutputParser()
)

# Example query
query = "Is the Cozy Comfort Pullover Set available in different colors?"

# Run the chain
result = llm_chain.run({"query": query})

# Print the result
print(result)


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


answer='Yes, it is available in multiple colors such as gray, navy, and burgundy.'


In [17]:
query_2 = "What sizes are available for the Cozy Comfort Pullover Set?"

# Run the chain
result_2 = llm_chain.run({"query": query_2})

# Print the result
print(result_2)

answer='The Cozy Comfort Pullover Set is available in sizes small, medium, large, and extra-large.'


In [18]:
query_3 = "What anoraks are available in green and size L?"

# Run the chain
result_3 = llm_chain.run({"query": query_3})

# Print the result
print(result_3)

answer='The Greenpoint Anorak and the Cascade Anorak are available in size L in green.'


#### LLM-Generated examples

In [19]:
from langchain.evaluation.qa import QAGenerateChain

In [20]:
example_gen_chain = QAGenerateChain.from_llm(ChatOpenAI())

In [21]:
llm_chain = LLMChain(llm=llm, prompt=prompt_template)

In [22]:
new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in data[:5]]
)

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:369: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


In [23]:
new_examples

[{'qa_pairs': {'query': "How should customers determine their size when ordering the Women's Campside Oxfords?",
   'answer': 'Customers should order their regular shoe size. For half sizes not offered, they should order up to the next whole size.'}},
 {'qa_pairs': {'query': 'What are the dimensions of the Small and Medium sizes of the Recycled Waterhog Dog Mat, Chevron Weave?',
   'answer': 'The Small size has dimensions of 18" x 28" and the Medium size has dimensions of 22.5" x 34.5".'}},
 {'qa_pairs': {'query': "What features make the Infant and Toddler Girls' Coastal Chill Swimsuit a practical and stylish choice for young children?",
   'answer': "The Infant and Toddler Girls' Coastal Chill Swimsuit features bright colors, ruffles, and exclusive whimsical prints. It is made of four-way-stretch and chlorine-resistant fabric that maintains its shape and resists snags. The UPF 50+ rated fabric provides the highest sun protection possible, blocking 98% of the sun's harmful rays. The sw

In [24]:
new_examples[0]

{'qa_pairs': {'query': "How should customers determine their size when ordering the Women's Campside Oxfords?",
  'answer': 'Customers should order their regular shoe size. For half sizes not offered, they should order up to the next whole size.'}}

In [25]:
new_examples[4]

{'qa_pairs': {'query': 'What technology makes the EcoFlex 3L Storm Pants more breathable and guarantees to keep you dry and comfortable?',
  'answer': 'The EcoFlex 3L Storm Pants feature the new TEK O2 technology, which offers the most breathability ever tested and ensures dryness and comfort during any activity and in any weather.'}}

In [26]:
data[0]

Document(metadata={'source': '/content/OutdoorClothingCatalog_1000.csv', 'row': 0}, page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.")

In [27]:
data[5]

Document(metadata={'source': '/content/OutdoorClothingCatalog_1000.csv', 'row': 5}, page_content=": 5\nname: Smooth Comfort Check Shirt, Slightly Fitted\ndescription: Our men's slightly fitted check shirt is the perfect choice for your wardrobe! Customers love how it fits right out of the dryer. Size & Fit: Slightly Fitted, Relaxed through the chest and sleeve with a slightly slimmer waist. Fabric & Care: 100% cotton poplin, with wrinkle-free performance that won't wash out. Our innovative TrueCool® fabric wicks moisture away from your skin and helps it dry quickly. Additional Features: Traditional styling with a button-down collar and a single patch pocket. Imported.")

In [28]:
d_flattened = [data['qa_pairs'] for data in new_examples]
d_flattened

[{'query': "How should customers determine their size when ordering the Women's Campside Oxfords?",
  'answer': 'Customers should order their regular shoe size. For half sizes not offered, they should order up to the next whole size.'},
 {'query': 'What are the dimensions of the Small and Medium sizes of the Recycled Waterhog Dog Mat, Chevron Weave?',
  'answer': 'The Small size has dimensions of 18" x 28" and the Medium size has dimensions of 22.5" x 34.5".'},
 {'query': "What features make the Infant and Toddler Girls' Coastal Chill Swimsuit a practical and stylish choice for young children?",
  'answer': "The Infant and Toddler Girls' Coastal Chill Swimsuit features bright colors, ruffles, and exclusive whimsical prints. It is made of four-way-stretch and chlorine-resistant fabric that maintains its shape and resists snags. The UPF 50+ rated fabric provides the highest sun protection possible, blocking 98% of the sun's harmful rays. The swimsuit also has crossover no-slip straps and

#### Combine examples

In [29]:
# examples += new_example
examples += d_flattened

In [30]:
examples[0]

{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
 'answer': 'Yes'}

**Querying the RetrievalQA chain with the questions provided in the examples**

In [32]:
qa.invoke(examples[0]["query"])



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
 'result': 'Yes, the Cozy Comfort Pullover Set does have side pockets.'}

In [33]:
qa.invoke(examples[1]["query"])



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?',
 'result': 'The Ultra-Lofty 850 Stretch Down Hooded Jacket is from the DownTek collection.'}

### Manual Evaluation - Fun part

In [34]:
import langchain
langchain.debug = True #enabling debug mode to be more verbose

In [35]:
qa.invoke(examples[0]["query"])

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Do the Cozy Comfort Pullover Set        have side pockets?"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "Do the Cozy Comfort Pullover Set        have side pockets?",
  "context": ": 73\nname: Cozy Cuddles Knit Pullover Set\ndescription: Perfect for lounging, this knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out. \n\nSize & Fit \nPants are Favorite Fit: Sits lower on the waist. \nRelaxed Fit: Our most generous fit sits farthest from the body. \n\nFabric & Care \nIn the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features \nRelaxed fit top with raglan sleeves and rounded hem. \nPull-on pants have 

{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
 'result': 'Yes, the Cozy Comfort Pullover Set does have side pockets.'}

In [36]:
# Turn off the debug mode
langchain.debug = False

### LLM assisted evaluation

In [37]:
examples += d_flattened

In [38]:
examples

[{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
  'answer': 'Yes'},
 {'query': 'What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?',
  'answer': 'The DownTek collection'},
 {'query': "How should customers determine their size when ordering the Women's Campside Oxfords?",
  'answer': 'Customers should order their regular shoe size. For half sizes not offered, they should order up to the next whole size.'},
 {'query': 'What are the dimensions of the Small and Medium sizes of the Recycled Waterhog Dog Mat, Chevron Weave?',
  'answer': 'The Small size has dimensions of 18" x 28" and the Medium size has dimensions of 22.5" x 34.5".'},
 {'query': "What features make the Infant and Toddler Girls' Coastal Chill Swimsuit a practical and stylish choice for young children?",
  'answer': "The Infant and Toddler Girls' Coastal Chill Swimsuit features bright colors, ruffles, and exclusive whimsical prints. It is made of four-way-stretch and chlor

In [39]:
predictions = qa.batch(examples)



> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.


In [40]:
predictions

[{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
  'answer': 'Yes',
  'result': 'Yes, the Cozy Comfort Pullover Set does have side pockets.'},
 {'query': 'What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?',
  'answer': 'The DownTek collection',
  'result': 'The Ultra-Lofty 850 Stretch Down Hooded Jacket is from the DownTek collection.'},
 {'query': "How should customers determine their size when ordering the Women's Campside Oxfords?",
  'answer': 'Customers should order their regular shoe size. For half sizes not offered, they should order up to the next whole size.',
  'result': "To determine the size when ordering the Women's Campside Oxfords, customers should refer to the specific size chart provided by the retailer. The size chart typically includes measurements for chest, waist, hips, and sometimes height to help customers choose the best-fitting size based on their body measurements. It's important to follow the size chart gu

In [41]:
from langchain.evaluation.qa import QAEvalChain

In [42]:
llm = ChatOpenAI(temperature=0)
eval_chain = QAEvalChain.from_llm(llm) #compare predicted answers to reference answers and provide a judgment or score on the accuracy and relevance of the predictions

In [46]:
graded_outputs = eval_chain.evaluate(examples, predictions)

In [47]:
graded_outputs

[{'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'}]

In [48]:
for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    # print("Predicted Grade: " + graded_outputs[i]['text'])
    print()

Example 0:
Question: Do the Cozy Comfort Pullover Set        have side pockets?
Real Answer: Yes
Predicted Answer: Yes, the Cozy Comfort Pullover Set does have side pockets.

Example 1:
Question: What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?
Real Answer: The DownTek collection
Predicted Answer: The Ultra-Lofty 850 Stretch Down Hooded Jacket is from the DownTek collection.

Example 2:
Question: How should customers determine their size when ordering the Women's Campside Oxfords?
Real Answer: Customers should order their regular shoe size. For half sizes not offered, they should order up to the next whole size.
Predicted Answer: To determine the size when ordering the Women's Campside Oxfords, customers should refer to the specific size chart provided by the retailer. The size chart typically includes measurements for chest, waist, hips, and sometimes height to help customers choose the best-fitting size based on their body measurements. It's important t

### Example 2 - **RAGAS**
One can also easily evaluate your QA chains with the metrics offered in RAGAS

In [49]:
from langchain_huggingface import HuggingFaceEmbeddings
loader = TextLoader('/content/nyc_text.txt')
index = VectorstoreIndexCreator(embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2", model_kwargs = {'device': 'cpu'})).from_loaders([loader])


llm = ChatOpenAI(temperature= 0)
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=index.vectorstore.as_retriever(),
    return_source_documents=True,
)

/usr/local/lib/python3.10/dist-packages/langchain/indexes/vectorstore.py:129: UserWarning: Using InMemoryVectorStore as the default vectorstore.This memory store won't persist data. You should explicitlyspecify a vectorstore when using VectorstoreIndexCreator
  warnings.warn(


In [50]:
# testing it out

question = "How did New York City get its name?"
result = qa_chain.invoke({"query": question})
result["result"]

'New York City was originally named New Amsterdam by Dutch colonists in 1626. When the city came under British control in 1664, it was renamed New York after King Charles II of England granted the lands to his brother, the Duke of York. The city has been continuously named New York since November 1674.'

In [51]:
# testing it out

question_lon = "How did London get its name?"
result_lon = qa_chain.invoke({"query": question_lon})
result_lon["result"]

"I don't know."

In [52]:
result

{'query': 'How did New York City get its name?',
 'result': 'New York City was originally named New Amsterdam by Dutch colonists in 1626. When the city came under British control in 1664, it was renamed New York after King Charles II of England granted the lands to his brother, the Duke of York. The city has been continuously named New York since November 1674.',
 'source_documents': [Document(id='4b43ed86-4c36-4edd-818d-0c1b99ff4fa7', metadata={'source': '/content/nyc_text.txt'}, page_content='The city and its metropolitan area constitute the premier gateway for legal immigration to the United States. As many as 800 languages are spoken in New York, making it the most linguistically diverse city in the world. New York City is home to more than 3.2 million residents born outside the U.S., the largest foreign-born population of any city in the world as of 2016.New York City traces its origins to a trading post founded on the southern tip of Manhattan Island by Dutch colonists in approxi

Now in order to evaluate the qa system we generated a few relevant questions. We've generated a few question for you but feel free to add any you want.

In [53]:
eval_questions = [
    "What is the population of New York City as of 2020?",
    "Which borough of New York City has the highest population?",
    "What is the economic significance of New York City?",
    "How did New York City get its name?",
    "What is the significance of the Statue of Liberty in New York City?",
    "When was New York City founded?", #new question
]

eval_answers = [
    "8,804,190",
    "Brooklyn",
    "New York City's economic significance is vast, as it serves as the global financial capital, housing Wall Street and major financial institutions. Its diverse economy spans technology, media, healthcare, education, and more, making it resilient to economic fluctuations. NYC is a hub for international business, attracting global companies, and boasts a large, skilled labor force. Its real estate market, tourism, cultural industries, and educational institutions further fuel its economic prowess. The city's transportation network and global influence amplify its impact on the world stage, solidifying its status as a vital economic player and cultural epicenter.",
    "New York City got its name when it came under British control in 1664. King Charles II of England granted the lands to his brother, the Duke of York, who named the city New York in his own honor.",
    "The Statue of Liberty in New York City holds great significance as a symbol of the United States and its ideals of liberty and peace. It greeted millions of immigrants who arrived in the U.S. by ship in the late 19th and early 20th centuries, representing hope and freedom for those seeking a better life. It has since become an iconic landmark and a global symbol of cultural diversity and freedom.",
    "New York City was founded in 1624.",  #new answer
]

examples = [
    {"query": q, "ground_truths": [eval_answers[i]]}
    for i, q in enumerate(eval_questions)
]

In [54]:
examples

[{'query': 'What is the population of New York City as of 2020?',
  'ground_truths': ['8,804,190']},
 {'query': 'Which borough of New York City has the highest population?',
  'ground_truths': ['Brooklyn']},
 {'query': 'What is the economic significance of New York City?',
  'ground_truths': ["New York City's economic significance is vast, as it serves as the global financial capital, housing Wall Street and major financial institutions. Its diverse economy spans technology, media, healthcare, education, and more, making it resilient to economic fluctuations. NYC is a hub for international business, attracting global companies, and boasts a large, skilled labor force. Its real estate market, tourism, cultural industries, and educational institutions further fuel its economic prowess. The city's transportation network and global influence amplify its impact on the world stage, solidifying its status as a vital economic player and cultural epicenter."]},
 {'query': 'How did New York City

#### Introducing RagasEvaluatorChain

`RagasEvaluatorChain` creates a wrapper around the metrics ragas provides (documented [here](https://github.com/explodinggradients/ragas/blob/main/docs/metrics.md)), making it easier to run these evaluation with langchain and langsmith.

The evaluator chain has the following APIs

- `__call__()`: call the `RagasEvaluatorChain` directly on the result of a QA chain.
- `evaluate()`: evaluate on a list of examples (with the input queries) and predictions (outputs from the QA chain).
- `evaluate_run()`: method implemented that is called by langsmith evaluators to evaluate langsmith datasets.

lets see each of them in action to learn more.

In [55]:
result = qa_chain.invoke({"query": eval_questions[1]})
result["result"]

'Manhattan (New York County) has the highest population density of any borough in New York City.'

In [56]:
key_mapping = {
    "query": "question",
    "result": "answer",
    "source_documents": "contexts"
}

result_updated = {}
for old_key, new_key in key_mapping.items():
    if old_key in result:
        result_updated[new_key] = result[old_key]


In [57]:
result_updated

{'question': 'Which borough of New York City has the highest population?',
 'answer': 'Manhattan (New York County) has the highest population density of any borough in New York City.',
 'contexts': [Document(id='c0b87d82-940e-46c1-818b-8c316097f9e4', metadata={'source': '/content/nyc_text.txt'}, page_content="New York City is the most populous city in the United States, with 8,804,190 residents incorporating more immigration into the city than outmigration since the 2010 United States census. More than twice as many people live in New York City as compared to Los Angeles, the second-most populous U.S. city; and New York has more than three times the population of Chicago, the third-most populous U.S. city. New York City gained more residents between 2010 and 2020 (629,000) than any other U.S. city, and a greater amount than the total sum of the gains over the same decade of the next four largest U.S. cities, Los Angeles, Chicago, Houston, and Phoenix, Arizona combined. New York City's 

In [58]:
result_updated['contexts']

[Document(id='c0b87d82-940e-46c1-818b-8c316097f9e4', metadata={'source': '/content/nyc_text.txt'}, page_content="New York City is the most populous city in the United States, with 8,804,190 residents incorporating more immigration into the city than outmigration since the 2010 United States census. More than twice as many people live in New York City as compared to Los Angeles, the second-most populous U.S. city; and New York has more than three times the population of Chicago, the third-most populous U.S. city. New York City gained more residents between 2010 and 2020 (629,000) than any other U.S. city, and a greater amount than the total sum of the gains over the same decade of the next four largest U.S. cities, Los Angeles, Chicago, Houston, and Phoenix, Arizona combined. New York City's population is about 44% of New York State's population, and about 39% of the population of the New York metropolitan area. The majority of New York City residents in 2020 (5,141,538, or 58.4%) were 

In [59]:
contexts_str_list = []

# Iterate through each Document in result_updated['contexts']
for document in result_updated['contexts']:
    # Append the page_content of each Document to contexts_str_list
    contexts_str_list.append(document.page_content)

# Join the list of strings with newline character
contexts_str = "\n".join(contexts_str_list)

# Assign the concatenated string back to result_updated['contexts']
result_updated['contexts'] = contexts_str



In [60]:
result_updated

{'question': 'Which borough of New York City has the highest population?',
 'answer': 'Manhattan (New York County) has the highest population density of any borough in New York City.',
 'contexts': "New York City is the most populous city in the United States, with 8,804,190 residents incorporating more immigration into the city than outmigration since the 2010 United States census. More than twice as many people live in New York City as compared to Los Angeles, the second-most populous U.S. city; and New York has more than three times the population of Chicago, the third-most populous U.S. city. New York City gained more residents between 2010 and 2020 (629,000) than any other U.S. city, and a greater amount than the total sum of the gains over the same decade of the next four largest U.S. cities, Los Angeles, Chicago, Houston, and Phoenix, Arizona combined. New York City's population is about 44% of New York State's population, and about 39% of the population of the New York metropol

**Comments**

The model answers the question 'Which borough of New York City has the highest population?' incorrectly. It answers that Manhattan has the highest population density, instead of total population.




In [61]:
!pip install --quiet --no-cache-dir recordclass

In [62]:
!pip install --quiet ragas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 5.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
gcsfs 2023.6.0 requires fsspec==2023.6.0, but you have fsspec 2024.5.0 which is incompatible.
torchaudio 2.3.0+cu121 requires torch==2.3.0, but you have torch 2.3.1 which is incompatible.
torchvision 0.18.0+cu121 requires torch==2.3.0, but you have torch 2.3.1 which is incompatible.


In [63]:
!pip install --quiet nest_asyncio


In [66]:
import nest_asyncio

nest_asyncio.apply()

In [67]:
from ragas.integrations.langchain import EvaluatorChain
# from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_relevancy,
    context_recall,
)

# create evaluation chains
faithfulness_chain   = EvaluatorChain(metric=faithfulness)
answer_rel_chain     = EvaluatorChain(metric=answer_relevancy)
context_rel_chain    = EvaluatorChain(metric=context_relevancy)
context_recall_chain = EvaluatorChain(metric=context_recall)

1. `__call__()`

Directly run the evaluation chain with the results from the QA chain. Do note that metrics like context_relevancy and faithfulness require the `source_documents` to be present.

In [68]:
# Recheck the result that we are going to validate.
result

{'query': 'Which borough of New York City has the highest population?',
 'result': 'Manhattan (New York County) has the highest population density of any borough in New York City.',
 'source_documents': [Document(id='c0b87d82-940e-46c1-818b-8c316097f9e4', metadata={'source': '/content/nyc_text.txt'}, page_content="New York City is the most populous city in the United States, with 8,804,190 residents incorporating more immigration into the city than outmigration since the 2010 United States census. More than twice as many people live in New York City as compared to Los Angeles, the second-most populous U.S. city; and New York has more than three times the population of Chicago, the third-most populous U.S. city. New York City gained more residents between 2010 and 2020 (629,000) than any other U.S. city, and a greater amount than the total sum of the gains over the same decade of the next four largest U.S. cities, Los Angeles, Chicago, Houston, and Phoenix, Arizona combined. New York Ci

**Faithfulness**

In [69]:
result_updated

{'question': 'Which borough of New York City has the highest population?',
 'answer': 'Manhattan (New York County) has the highest population density of any borough in New York City.',
 'contexts': "New York City is the most populous city in the United States, with 8,804,190 residents incorporating more immigration into the city than outmigration since the 2010 United States census. More than twice as many people live in New York City as compared to Los Angeles, the second-most populous U.S. city; and New York has more than three times the population of Chicago, the third-most populous U.S. city. New York City gained more residents between 2010 and 2020 (629,000) than any other U.S. city, and a greater amount than the total sum of the gains over the same decade of the next four largest U.S. cities, Los Angeles, Chicago, Houston, and Phoenix, Arizona combined. New York City's population is about 44% of New York State's population, and about 39% of the population of the New York metropol

In [72]:
eval_result = faithfulness_chain(result_updated)
eval_result['faithfulness'] #key updated

0.5

High faithfulness_score means that there are exact consistency between the source documents and the answer.

You can check lower faithfulness scores by changing the result (answer from LLM) or source_documents to something else.

In [80]:
fake_result = result_updated.copy()
fake_result["answer"] = "we are the champions"
fake_result["contexts"] = "Queen"

fake_result

{'question': 'Which borough of New York City has the highest population?',
 'answer': 'we are the champions',
 'contexts': 'Queen'}

In [81]:
eval_result = faithfulness_chain(fake_result)
eval_result["faithfulness"]

nan

**Context Relevancy**

In [ ]:
result_updated

**Adding the ground truth to the results dictionary**

In [88]:
# Find matching ground truths
for item in examples:
    if item['query'] == result_updated['question']:
        result_updated['ground_truth'] = item['ground_truths']
        break

if 'ground_truths' in result_updated:
    del result_updated['ground_truths']

result_updated

{'question': 'Which borough of New York City has the highest population?',
 'answer': 'Manhattan (New York County) has the highest population density of any borough in New York City.',
 'contexts': "New York City is the most populous city in the United States, with 8,804,190 residents incorporating more immigration into the city than outmigration since the 2010 United States census. More than twice as many people live in New York City as compared to Los Angeles, the second-most populous U.S. city; and New York has more than three times the population of Chicago, the third-most populous U.S. city. New York City gained more residents between 2010 and 2020 (629,000) than any other U.S. city, and a greater amount than the total sum of the gains over the same decade of the next four largest U.S. cities, Los Angeles, Chicago, Houston, and Phoenix, Arizona combined. New York City's population is about 44% of New York State's population, and about 39% of the population of the New York metropol

In [89]:
eval_result = context_recall_chain(result_updated)
eval_result["context_recall"]

0.5

High context_recall_score means that the ground truth is present in the source documents.

You can check lower context recall scores by changing the source_documents to something else.

In [97]:
from langchain.schema import Document

# Create the fake result with a Document
fake_result = result_updated.copy()
fake_result["contexts"] = [Document(page_content="I love Christmas")]

# Extract page_content from each Document in the list
contexts = [doc.page_content for doc in fake_result['contexts']]

# Update the fake_result with the extracted contexts
fake_result['contexts'] = contexts

In [98]:
fake_result

{'question': 'Which borough of New York City has the highest population?',
 'answer': 'Manhattan (New York County) has the highest population density of any borough in New York City.',
 'contexts': ['I love Christmas'],
 'ground_truth': ['Brooklyn']}

In [99]:
eval_result = context_recall_chain(fake_result)
eval_result["context_recall"]

0.0

2. `evaluate()`

Evaluate a list of inputs/queries and the outputs/predictions from the QA chain.

In [100]:
# run the queries as a batch for efficiency
predictions = qa_chain.batch(examples)

In [125]:
# evaluate
from langchain.evaluation.qa import QAEvalChain

for example in examples:
    if 'ground_truths' in example:
        example['answer'] = example.pop('ground_truths')

examples

[{'query': 'What is the population of New York City as of 2020?',
  'answer': ['8,804,190']},
 {'query': 'Which borough of New York City has the highest population?',
  'answer': ['Brooklyn']},
 {'query': 'What is the economic significance of New York City?',
  'answer': ["New York City's economic significance is vast, as it serves as the global financial capital, housing Wall Street and major financial institutions. Its diverse economy spans technology, media, healthcare, education, and more, making it resilient to economic fluctuations. NYC is a hub for international business, attracting global companies, and boasts a large, skilled labor force. Its real estate market, tourism, cultural industries, and educational institutions further fuel its economic prowess. The city's transportation network and global influence amplify its impact on the world stage, solidifying its status as a vital economic player and cultural epicenter."]},
 {'query': 'How did New York City get its name?',
  'a

In [127]:
predictions

[{'query': 'What is the population of New York City as of 2020?',
  'ground_truths': ['8,804,190'],
  'result': 'The population of New York City as of 2020 is 8,804,190 residents.',
  'source_documents': [Document(id='c0b87d82-940e-46c1-818b-8c316097f9e4', metadata={'source': '/content/nyc_text.txt'}, page_content="New York City is the most populous city in the United States, with 8,804,190 residents incorporating more immigration into the city than outmigration since the 2010 United States census. More than twice as many people live in New York City as compared to Los Angeles, the second-most populous U.S. city; and New York has more than three times the population of Chicago, the third-most populous U.S. city. New York City gained more residents between 2010 and 2020 (629,000) than any other U.S. city, and a greater amount than the total sum of the gains over the same decade of the next four largest U.S. cities, Los Angeles, Chicago, Houston, and Phoenix, Arizona combined. New York C

In [126]:

print("evaluating...")
r = faithfulness_chain.evaluate(examples, predictions)
r

evaluating...


[{'results': 'CORRECT'},
 {'results': 'INCORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'}]

In [134]:

# evaluate context recall
print("evaluating...")
r = context_recall_chain.run_evaluation(examples, predictions)
r

evaluating...


AttributeError: 'EvaluatorChain' object has no attribute 'run_evaluation'